In [137]:
%%configure -f
{ "conf":{
"spark.pyspark.python": "python3",
"spark.pyspark.virtualenv.enabled": "true",
"spark.pyspark.virtualenv.type":"native",
"spark.pyspark.virtualenv.bin.path":"/usr/bin/virtualenv"
}}

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
4,application_1589733083011_0005,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
4,application_1589733083011_0005,pyspark,idle,Link,Link,✔


In [37]:
##Datos descargados de la segunda fuente
##La tercera fuenta esta desactualizada y no tiene para exportar los datos
df=spark.read.csv('s3://trabajo3-dataset/colombia/Casos_positivos_de_COVID-19_en_Colombia(insGov).csv',inferSchema=True,header=True)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [38]:
##Columnas
df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- ID de caso: integer (nullable = true)
 |-- Fecha de notificación: timestamp (nullable = true)
 |-- Codigo DIVIPOLA: integer (nullable = true)
 |-- Ciudad de ubicación: string (nullable = true)
 |-- Departamento o Distrito : string (nullable = true)
 |-- atención: string (nullable = true)
 |-- Edad: integer (nullable = true)
 |-- Sexo: string (nullable = true)
 |-- Tipo: string (nullable = true)
 |-- Estado: string (nullable = true)
 |-- País de procedencia: string (nullable = true)
 |-- FIS: string (nullable = true)
 |-- Fecha de muerte: string (nullable = true)
 |-- Fecha diagnostico: timestamp (nullable = true)
 |-- Fecha recuperado: string (nullable = true)
 |-- fecha reporte web: timestamp (nullable = true)

In [39]:
#Numero de casos
df.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

14216

In [40]:
#Fase de limpieza de datos
#Analizo el campo atención
df.groupBy('atención').count().show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------+-----+
|    atención|count|
+------------+-----+
|Hospital UCI|  136|
|   Fallecido|  546|
|        Casa| 9292|
|    Hospital|  769|
|         N/A|   13|
|  Recuperado| 3460|
+------------+-----+

In [41]:
#Analizo el campo Estado
df.groupBy('Estado').count().show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------+-----+
|      Estado|count|
+------------+-----+
|        null|   15|
|   Fallecido|  544|
|   Fallecdio|    1|
|    Moderado|  806|
|        leve|   11|
|Asintomático| 1717|
|        Leve|10987|
|       Grave|  135|
+------------+-----+

In [42]:
#Analizo el campo Tipo
df.groupBy('Tipo').count().show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+-----+
|       Tipo|count|
+-----------+-----+
|Relacionado| 4137|
|  Importado|  840|
| En Estudio|    2|
| En estudio| 9237|
+-----------+-----+

In [43]:
#Analizo el campo País de procedencia
df.groupBy('País de procedencia').count().show(50)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+-----+
| País de procedencia|count|
+--------------------+-----+
|           COILOMBIA|    1|
|              Canadá|    4|
|               Suiza|    1|
|               Kenia|    1|
|             Croacia|    1|
|              Egipto|   11|
|              Italia|    9|
|             Francia|   16|
|              España|  259|
|           Argentina|    4|
|             Ecuador|   50|
|             Bélgica|    1|
|         Puerto Rico|   13|
|               Chile|    4|
|              Panamá|   34|
|            Alemania|   10|
|             Turquía|   28|
|                Cuba|    4|
|Emiratos Árabes U...|    2|
|              Grecia|    4|
|           Venezuela|    4|
|              Israel|    6|
|      Arabia Saudita|    1|
|República Dominicana|   10|
|Estados Unidos de...|  217|
|             Uruguay|    1|
|               Aruba|    3|
|           Guatemala|    1|
|San Martín (Francia)|    7|
|            Honduras|    1|
|             Jamaica|   16|
|         SANT

In [44]:
# 1. Remplazo en 'atencion' los valores' N/A' por 'fallecido', cuando existe fecha de muerte
# 2. Remplazo en 'Estado' los valores 'null' por 'fallecido', cuando existe fecha de muerte
# 3. Remplazo en 'Estado' los valores 'fallecdio' por 'fallecido'
# 4. Remplazo en 'Estado' los valores 'leve' por 'Leve'
# 5. Remplazo en 'Tipo' los valores 'En Estudio' por 'En estudio'
# 6. Remplazo en 'País de procedencia' los valores 'COILOMBIA', 'COLOMBIANA' y 'SANTA MARTA' por 'Colombia'
# 7. Remplazo en 'País de procedencia' los valores 'San Martín (Francia)' por 'Francia'

from pyspark.sql import functions as F

df = df.withColumn("atención", F.when((F.col("atención")=="N/A")&(F.col("Fecha de muerte")!="-   -"), "Fallecido").otherwise(F.col("atención")))

df = df.withColumn("Estado", F.when((F.col("Estado").isNull())&(F.col("Fecha de muerte")!="-   -"), "Fallecido").otherwise(F.col("Estado")))
df = df.withColumn("Estado", F.when(F.col("Estado")=="Fallecdio", "Fallecido").otherwise(F.col("Estado")))
df = df.withColumn("Estado", F.when(F.col("Estado")=="leve", "Leve").otherwise(F.col("Estado")))

df = df.withColumn("Tipo", F.when(F.col("Tipo")=="En Estudio", "En estudio").otherwise(F.col("Tipo")))

df = df.withColumn("País de procedencia", F.when((F.col("País de procedencia")=="COILOMBIA")|(F.col("País de procedencia")=="COLOMBIANA")|(F.col("País de procedencia")=="SANTA MARTA"), "Colombia").otherwise(F.col("País de procedencia")))
df = df.withColumn("País de procedencia", F.when(F.col("País de procedencia")=="San Martín (Francia)", "Francia").otherwise(F.col("País de procedencia")))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [45]:
#Fase de analisis exploratorio
# Cambio el tipo de dato "string" a "timestamp" de la columna FIS
# Calculo la diferencia de tiempo entre "Fecha de notificacion" y "FIS"
# Agrego la nueva columna "Dias de exposicion" al dataFrame
# Dias de exposicion: el tiempo que pasa desde de que una persona tiene sintomas, hasta que los notifica al sistema de salud
# Este campo es importante, ya que el tiempo que pasa el paciente sin notificar los sintomas es posible que pueda exparsir el virus
# En caso de que el campo sea "Asintomatico" en la columna "FIS", es imposible saber cuanto tiempo el paciente pudo exparsir el virus
 

df = df.withColumn('FIS', F.col('FIS').cast('timestamp'))
diff_secs_col = F.col("Fecha de notificación").cast("long") - F.col("FIS").cast("long")
df = df.withColumn( "Dias de exposicion", diff_secs_col/86400  )

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [46]:
# Agrego 2 columnas temporales "inicio" y "fin"
# inicio: fecha en la que el virus aparece en el paciente. se calcula a partir de la fecha de inicio de sintomas(FIS) 
# y en caso de ser Asintomatico, se calcula a partir de la fecha de notificacion.
# fin: fecha en la que el virus desaparece del paciente. se calcula con la fecha de muerte o fecha de recuperacion del paciente.
# en caso de que ambas fechas sean "null", significa que el paciente sigue infectado

df = df.withColumn("inicio", F.when(F.col("FIS")=="Asintomático", F.col('Fecha de notificación')).otherwise(F.col("FIS").cast('timestamp')))
df = df.withColumn("fin", F.when(F.col("Fecha de muerte")=="-   -", F.col('Fecha recuperado').cast('timestamp')).otherwise(F.col("Fecha de muerte").cast('timestamp')))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [47]:
# Agrego la nueva columna "Duracion de infeccion"
# Duracion de infeccion: es el tiempo en dias que un paciente dura infectado con el virus.

diff_secs_col = F.col("fin").cast("long") - F.col("inicio").cast("long")
df = df.withColumn( "Duracion de infeccion", diff_secs_col/86400  )

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [48]:
# Elimino las columnas ['ID de caso', 'Codigo DIVIPOLA', 'fecha reporte web'. 'fin', 'inicio']
# ya que no son relevantes para el analisis
df = df.select('Fecha de notificación', 'Ciudad de ubicación', 'Departamento o Distrito ', 'atención', 'Edad', 'Sexo', 'Tipo', 'Estado', 'País de procedencia', 'FIS', 'Fecha de muerte', 'Fecha diagnostico', 'Fecha recuperado', 'Dias de exposicion', 'Duracion de infeccion')
df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- Fecha de notificación: timestamp (nullable = true)
 |-- Ciudad de ubicación: string (nullable = true)
 |-- Departamento o Distrito : string (nullable = true)
 |-- atención: string (nullable = true)
 |-- Edad: integer (nullable = true)
 |-- Sexo: string (nullable = true)
 |-- Tipo: string (nullable = true)
 |-- Estado: string (nullable = true)
 |-- País de procedencia: string (nullable = true)
 |-- FIS: timestamp (nullable = true)
 |-- Fecha de muerte: string (nullable = true)
 |-- Fecha diagnostico: timestamp (nullable = true)
 |-- Fecha recuperado: string (nullable = true)
 |-- Dias de exposicion: double (nullable = true)
 |-- Duracion de infeccion: double (nullable = true)

In [50]:
############## Grabo los resultados en mi bucket de S3 #####################
# Direccion del bucket
write_uri='s3://trabajo3-dataset/procesado/colombia'
# Almacenamiento del dataFrame como un archivo csv
df.coalesce(1).write.format("csv").option("header","true").save(write_uri)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [138]:
## Ahora los datasets del mundo 
## cargamos el de casos confirmados, 
dfConfirmados=spark.read.csv('s3://trabajo3-dataset/mundial/time_series_covid19_confirmed_global.csv',inferSchema=True,header=True)
dfMuertos=spark.read.csv('s3://trabajo3-dataset/mundial/time_series_covid19_deaths_global.csv',inferSchema=True,header=True)
dfRecuperados=spark.read.csv('s3://trabajo3-dataset/mundial/time_series_covid19_recovered_global.csv',inferSchema=True,header=True)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [141]:
# Calculo el total de casos, muertos y recuperados de cada pais sumando el total de todas las fechas juntas

totalCasos = dfConfirmados.groupBy('Country/Region').agg(F.sum('5/8/20').alias('casos')).orderBy('casos', ascending=False).collect()
totalMuertes = dfMuertos.groupBy('Country/Region').agg(F.sum('5/8/20').alias('muertes')).orderBy('muertes', ascending=False).collect()
totalRecuperados = dfRecuperados.groupBy('Country/Region').agg(F.sum('5/8/20').alias('recuperados')).orderBy('recuperados', ascending=False).collect()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [142]:
# Creo un arreglo con el nombre de cada pais
paises = [totalCasos[n][0] for n in range(len(totalRecuperados))]

# Transformo las listas 'totalCasos', 'totalMuertes' y 'totalRecuperados' en diccionarios
# de la forma {nombrePais: cantidad}
casosDict = { totalCasos[n][0] : totalCasos[n][1] for n in range(len(totalCasos)) }
muertesDict = { totalMuertes[n][0] : totalMuertes[n][1] for n in range(len(totalMuertes)) }
recuperadosDict = { totalRecuperados[n][0] : totalRecuperados[n][1] for n in range(len(totalRecuperados)) }

# Creo una matriz de con las columnas [NombrePais, totalCasos, totalMuertes, totalRecuperados]
matrix = []
for pais in paises:
    matrix.append([pais, casosDict[pais], muertesDict[pais], recuperadosDict[pais]])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [143]:
# Creo en dataframe desde la matriz
columns=["Pais", "Casos", "Fallecidos", "Recuperados"]

df=spark.createDataFrame(matrix,schema=columns)
df.show(187)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+-------+----------+-----------+
|                Pais|  Casos|Fallecidos|Recuperados|
+--------------------+-------+----------+-----------+
|                  US|1283929|     77180|     198993|
|               Spain| 222857|     26299|     131148|
|               Italy| 217185|     30201|      99023|
|      United Kingdom| 212629|     31316|        997|
|              Russia| 187859|      1723|      26608|
|              France| 176202|     26233|      55892|
|             Germany| 170588|      7510|     141700|
|              Brazil| 146894|     10017|      59297|
|              Turkey| 135569|      3689|      86396|
|                Iran| 104691|      6541|      83837|
|               China|  83976|      4637|      78993|
|              Canada|  67674|      4697|      30239|
|                Peru|  61847|      1714|      19012|
|               India|  59695|      1985|      17887|
|             Belgium|  52011|      8521|      13201|
|         Netherlands|  4229

In [144]:
# Agrego una nueva columna 'Infectados'
# Infectados: son los pacientes que aun siguen con el virus (no se han recuperado o fallecido)

diff_secs_col = F.col("Casos") - F.col("Fallecidos") - F.col("Recuperados")
df = df.withColumn( "Infectados", diff_secs_col  )
df.show(187)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+-------+----------+-----------+----------+
|                Pais|  Casos|Fallecidos|Recuperados|Infectados|
+--------------------+-------+----------+-----------+----------+
|                  US|1283929|     77180|     198993|   1007756|
|               Spain| 222857|     26299|     131148|     65410|
|               Italy| 217185|     30201|      99023|     87961|
|      United Kingdom| 212629|     31316|        997|    180316|
|              Russia| 187859|      1723|      26608|    159528|
|              France| 176202|     26233|      55892|     94077|
|             Germany| 170588|      7510|     141700|     21378|
|              Brazil| 146894|     10017|      59297|     77580|
|              Turkey| 135569|      3689|      86396|     45484|
|                Iran| 104691|      6541|      83837|     14313|
|               China|  83976|      4637|      78993|       346|
|              Canada|  67674|      4697|      30239|     32738|
|                Peru|  6

In [145]:
############## Grabo los resultados en mi bucket de S3 #####################
# Direccion del bucket
write_uri='s3://trabajo3-dataset/procesado/global'
# Almacenamiento del dataFrame como un archivo csv
df.coalesce(1).write.format("csv").option("header","true").save(write_uri)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…